Airlines 延误数据集 BTTWD 实验

本 notebook 按步骤运行：环境准备 → 加载配置 → 读取数据 → 预处理 → 桶树划分 → 基线与 BTTWD 实验 → 桶级分析。


In [1]:
# 步骤0：环境与路径设置
import os, sys
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

# 将项目根目录加入路径，便于导入 bttwdlib
root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
if root_path not in sys.path:
    sys.path.append(root_path)

from bttwdlib import (
    load_yaml_cfg,
    show_cfg,
    load_dataset,
    prepare_features_and_labels,
    BucketTree,
    run_holdout_experiment,
    run_kfold_experiments,
    log_info,
    set_global_seed,
)

cfg_path = os.path.join(root_path, 'configs', 'airlines_delay.yaml')
cfg = load_yaml_cfg(cfg_path)
set_global_seed(cfg.get('SEED', {}).get('global_seed', 42))
log_info('【步骤0摘要】环境准备完毕，路径与随机种子已设置。')


【INFO】【2025-11-27 13:50:32】【配置加载】已读取 e:\yan\组\三支决策\机器学习\BT_TWD\configs\airlines_delay.yaml
【INFO】【2025-11-27 13:50:36】【步骤0摘要】环境准备完毕，路径与随机种子已设置。


In [2]:
# 步骤1：加载配置
show_cfg(cfg)
log_info('【步骤1摘要】配置文件加载完成，关键参数检查通过。')


【INFO】【2025-11-27 13:50:36】【配置-数据】数据集=airlines_delay_1m, k折=5, 目标列=DepDelay, 正类="1"
【INFO】【2025-11-27 13:50:36】【配置-BTTWD】阈值模式=None, 全局模型=xgb, 桶内模型=none, 后验估计器(兼容字段)=logreg
【INFO】【2025-11-27 13:50:36】【配置-基线】LogReg启用=False, RandomForest启用=False, KNN启用=False, XGBoost启用=True
【INFO】【2025-11-27 13:50:36】【步骤1摘要】配置文件加载完成，关键参数检查通过。


In [3]:
# 步骤2：加载原始数据
df_raw, target_col_model = load_dataset(cfg)  # 这里返回的是用于建模的标签列，例如 "label"

display(df_raw.head())
print("用于建模的标签列:", target_col_model)

# 1）画 0/1 标签（延误/不延误）的比例
class_counts = df_raw[target_col_model].value_counts(normalize=True)
ax = class_counts.plot(kind='bar', title='延误 vs 未延误比例')
plt.ylabel('比例')

fig_path = os.path.join(root_path, cfg['OUTPUT']['figs_dir'], 'class_distribution.png')
os.makedirs(os.path.dirname(fig_path), exist_ok=True)
plt.savefig(fig_path, bbox_inches='tight')
plt.close()

# 2）如果想看原始 DepDelay 的分布，可以另外单独分析：
raw_target_col = cfg['DATA']['target_col']  # 这里是 "DepDelay"
print("原始目标列:", raw_target_col)
print(df_raw[raw_target_col].describe())

log_info('【步骤2摘要】Airlines 原始数据加载与基本统计完成。')


【INFO】【2025-11-27 13:50:53】【数据加载】ARFF 文件 ..\data\airline\airlines_train_regression_1000000.arff 已读取，含 1000000 条记录，10 列
【INFO】【2025-11-27 13:50:53】【目标变换】已按阈值 15.0 生成二分类标签列 label，正类取 > 15.0
【INFO】【2025-11-27 13:50:53】【数据集信息】名称=airlines_delay_1m，样本数=1000000，目标列=label，正类比例=15.59%


,DepDelay,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,UniqueCarrier,Origin,Dest,Distance,label
0,8.0,10.0,11.0,7.0,1300.0,1535.0,AA,LAX,HNL,2556.0,0
1,-3.0,10.0,10.0,6.0,2035.0,2110.0,AA,OGG,HNL,100.0,0
2,6.0,10.0,26.0,1.0,1200.0,1446.0,AA,JFK,LAX,2475.0,0
3,1.0,10.0,9.0,5.0,1145.0,1512.0,AA,JFK,SFO,2586.0,0
4,0.0,10.0,16.0,5.0,930.0,1149.0,AA,SFO,HNL,2399.0,0


用于建模的标签列: label
原始目标列: DepDelay
count    1000000.000000
mean           8.191935
std           28.877186
min        -1197.000000
25%           -3.000000
50%            0.000000
75%            7.000000
max         2119.000000
Name: DepDelay, dtype: float64
【INFO】【2025-11-27 13:50:53】【步骤2摘要】Airlines 原始数据加载与基本统计完成。


In [4]:
# 步骤3：预处理与特征工程
X, y, meta = prepare_features_and_labels(df_raw, cfg)
log_info(f'【预处理】编码特征维度={X.shape[1]}，样本数={X.shape[0]}')
log_info(f"【步骤3摘要】特征预处理完成：连续={len(meta['continuous_cols'])}，类别={len(meta['categorical_cols'])}，编码维度={X.shape[1]}。")


【INFO】【2025-11-27 13:50:54】【预处理】连续特征=6个，类别特征=3个
【INFO】【2025-11-27 13:50:57】【预处理】编码后维度=755
【INFO】【2025-11-27 13:50:57】【预处理】编码特征维度=755，样本数=1000000
【INFO】【2025-11-27 13:50:57】【步骤3摘要】特征预处理完成：连续=6，类别=3，编码维度=755。


In [5]:
# 步骤4：构建桶树并检查划分
feature_df_for_bucket = df_raw.drop(columns=[cfg['DATA']['target_col']])
bucket_tree = BucketTree(cfg['BTTWD']['bucket_levels'], feature_names=feature_df_for_bucket.columns.tolist())
bucket_ids_full = bucket_tree.assign_buckets(feature_df_for_bucket)
bucket_df = bucket_ids_full.value_counts().reset_index()
bucket_df.columns = ['bucket_id', 'count']
bucket_df['pos_rate'] = df_raw.groupby(bucket_ids_full)[cfg['DATA']['target_col']].apply(
    lambda s: (s == cfg['DATA']['positive_label']).mean()
).values

display(bucket_df.head())
bucket_df.set_index('bucket_id')['count'].plot(kind='bar', figsize=(12,4), title='桶样本数分布')
fig_bucket = os.path.join(root_path, cfg['OUTPUT']['figs_dir'], 'bucket_metrics_bar.png')
plt.savefig(fig_bucket, bbox_inches='tight')
plt.close()
log_info(f'【步骤4摘要】桶树划分完成，共有 {bucket_ids_full.nunique()} 个叶子桶。')


【INFO】【2025-11-27 13:51:00】【桶树】已为样本生成桶ID，共 7200 个组合


,bucket_id,count,pos_rate
0,L1_UniqueCarrier=DL|L2_Origin=ATL|L4_Distance=...,7562,0.045455
1,L1_UniqueCarrier=DL|L2_Origin=ATL|L4_Distance=...,6048,0.080000
2,L1_UniqueCarrier=WN|L2_Origin=OTHER|L4_Distanc...,6047,0.045714
3,L1_UniqueCarrier=WN|L2_Origin=OTHER|L4_Distanc...,5923,0.031250
4,L1_UniqueCarrier=AA|L2_Origin=DFW|L4_Distance=...,5600,0.090909


【INFO】【2025-11-27 13:55:49】【步骤4摘要】桶树划分完成，共有 7200 个叶子桶。


In [6]:
# 步骤5：运行基线模型实验占位
# 基线部分在 run_kfold_experiments 内统一调度（仅在 use_kfold=True 时执行）
log_info('【步骤5】基线模型将在交叉验证模式中一并运行。')
log_info('【步骤5摘要】基线模型性能将作为后续对比基准。')


【INFO】【2025-11-27 13:55:49】【步骤5】基线模型将在交叉验证模式中一并运行。
【INFO】【2025-11-27 13:55:49】【步骤5摘要】基线模型性能将作为后续对比基准。


In [7]:
# 步骤6：运行 BTTWD 实验（k 折或单次留出）
use_kfold_raw = cfg.get('DATA', {}).get('use_kfold', False)
if isinstance(use_kfold_raw, str):
    use_kfold = use_kfold_raw.strip().lower() in ['true', '1', 'yes']
else:
    use_kfold = bool(use_kfold_raw)

if use_kfold:
    log_info('【步骤6】检测到 use_kfold=True，进入 k 折实验。')
    results = run_kfold_experiments(X, y, feature_df_for_bucket, cfg)
    summary_df = pd.read_csv(os.path.join(root_path, cfg['OUTPUT']['results_dir'], 'metrics_kfold_summary.csv'))
    display(summary_df)
    summary_df.plot(x='model', kind='bar', figsize=(8,4), title='模型指标对比')
    fig_compare = os.path.join(root_path, cfg['OUTPUT']['figs_dir'], 'metrics_compare.png')
    plt.savefig(fig_compare, bbox_inches='tight')
    plt.close()
    log_info('【步骤6摘要】BTTWD 与基线的 k 折结果已生成并保存。')
else:
    log_info('【步骤6】use_kfold=False，执行单次留出验证流程。')
    holdout_metrics = run_holdout_experiment(X, y, feature_df_for_bucket, cfg)
    display(pd.DataFrame(holdout_metrics))
    log_info('【步骤6摘要】单次留出验证完成，指标已列出。')


【INFO】【2025-11-27 13:55:49】【步骤6】检测到 use_kfold=True，进入 k 折实验。
【INFO】【2025-11-27 13:55:49】【基线-XGB】使用模型自定义阈值=0.200（per_model 模式）


C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:158: UserWarning: [13:56:24] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:158: UserWarning: [13:57:56] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:158: UserWarning: [13:59:29] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.



【INFO】【2025-11-27 14:02:54】【基线-XGB】整体指标：AUC_mean=0.685, AUC_std=0.001, BAC_mean=0.621, BAC_std=0.001, BND_ratio_mean=0.000, BND_ratio_std=0.000, F1_mean=0.348, F1_std=0.001, Kappa_mean=0.189, Kappa_std=0.001, MCC_mean=0.200, MCC_std=0.002, POS_Coverage_mean=nan, POS_Coverage_std=nan, Precision_mean=0.277, Precision_std=0.001, Recall_mean=0.468, Recall_std=0.002, Regret_mean=0.605, Regret_std=0.002
【INFO】【2025-11-27 14:02:55】【K折实验】正在执行第 1/5 折...
【INFO】【2025-11-27 14:03:29】【桶树】已为样本生成桶ID，共 6207 个组合


C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:158: UserWarning: [14:03:55] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


【INFO】【2025-11-27 14:04:23】【BTTWD】全局模型训练完成，用于兜底预测
[INFO][BT][2025-11-27 14:04:57] 桶 L1_UniqueCarrier=AA 子桶样本不足（最小子桶 n=68 < 200），不再细分
[INFO][BT][2025-11-27 14:04:57] 桶 L1_UniqueCarrier=AS 子桶样本不足（最小子桶 n=2 < 200），不再细分
[INFO][BT][2025-11-27 14:04:57] 桶 L1_UniqueCarrier=B6 子桶样本不足（最小子桶 n=2 < 200），不再细分
[INFO][BT][2025-11-27 14:04:57] 桶 L1_UniqueCarrier=CO 子桶样本不足（最小子桶 n=8 < 200），不再细分
[INFO][BT][2025-11-27 14:04:57] 桶 L1_UniqueCarrier=DL 子桶样本不足（最小子桶 n=95 < 200），不再细分
[INFO][BT][2025-11-27 14:04:57] 桶 L1_UniqueCarrier=EV 子桶样本不足（最小子桶 n=2 < 200），不再细分
[INFO][BT][2025-11-27 14:04:57] 桶 L1_UniqueCarrier=FL 子桶样本不足（最小子桶 n=13 < 200），不再细分
[INFO][BT][2025-11-27 14:04:57] 桶 L1_UniqueCarrier=HP 子桶样本不足（最小子桶 n=8 < 200），不再细分
[INFO][BT][2025-11-27 14:04:57] 桶 L1_UniqueCarrier=MQ 子桶样本不足（最小子桶 n=5 < 200），不再细分
[INFO][BT][2025-11-27 14:04:57] 桶 L1_UniqueCarrier=NW 子桶样本不足（最小子桶 n=4 < 200），不再细分
[INFO][BT][2025-11-27 14:04:57] 桶 L1_UniqueCarrier=OH 子桶样本不足（最小子桶 n=1 < 200），不再细分
[INFO][BT][2025-11-27 14:04:57] 桶 L1_UniqueCa

...

【INFO】【2025-11-27 14:10:49】【BTTWD】全局模型训练完成，用于兜底预测
[INFO][BT][2025-11-27 14:11:05] 桶 L1_UniqueCarrier=AA 子桶样本不足（最小子桶 n=66 < 200），不再细分
[INFO][BT][2025-11-27 14:11:05] 桶 L1_UniqueCarrier=AS 子桶样本不足（最小子桶 n=1 < 200），不再细分
[INFO][BT][2025-11-27 14:11:05] 桶 L1_UniqueCarrier=B6 子桶样本不足（最小子桶 n=2 < 200），不再细分
[INFO][BT][2025-11-27 14:11:05] 桶 L1_UniqueCarrier=CO 子桶样本不足（最小子桶 n=4 < 200），不再细分
[INFO][BT][2025-11-27 14:11:05] 桶 L1_UniqueCarrier=DL 子桶样本不足（最小子桶 n=95 < 200），不再细分
[INFO][BT][2025-11-27 14:11:05] 桶 L1_UniqueCarrier=EV 子桶样本不足（最小子桶 n=2 < 200），不再细分
[INFO][BT][2025-11-27 14:11:06] 桶 L1_UniqueCarrier=FL 子桶样本不足（最小子桶 n=11 < 200），不再细分
[INFO][BT][2025-11-27 14:11:06] 桶 L1_UniqueCarrier=HP 子桶样本不足（最小子桶 n=9 < 200），不再细分
[INFO][BT][2025-11-27 14:11:06] 桶 L1_UniqueCarrier=MQ 子桶样本不足（最小子桶 n=5 < 200），不再细分
[INFO][BT][2025-11-27 14:11:06] 桶 L1_UniqueCarrier=NW 子桶样本不足（最小子桶 n=4 < 200），不再细分
[INFO][BT][2025-11-27 14:11:06] 桶 L1_UniqueCarrier=OH 子桶样本不足（最小子桶 n=1 < 200），不再细分
[INFO][BT][2025-11-27 14:11:06] 桶 L1_UniqueCa

C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:158: UserWarning: [14:12:19] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


【INFO】【2025-11-27 14:12:49】【BTTWD】全局模型训练完成，用于兜底预测
[INFO][BT][2025-11-27 14:13:06] 桶 L1_UniqueCarrier=AA 子桶样本不足（最小子桶 n=63 < 200），不再细分
[INFO][BT][2025-11-27 14:13:06] 桶 L1_UniqueCarrier=AS 子桶样本不足（最小子桶 n=1 < 200），不再细分
[INFO][BT][2025-11-27 14:13:06] 桶 L1_UniqueCarrier=B6 子桶样本不足（最小子桶 n=2 < 200），不再细分
[INFO][BT][2025-11-27 14:13:06] 桶 L1_UniqueCarrier=CO 子桶样本不足（最小子桶 n=8 < 200），不再细分
[INFO][BT][2025-11-27 14:13:06] 桶 L1_UniqueCarrier=DL 子桶样本不足（最小子桶 n=90 < 200），不再细分
[INFO][BT][2025-11-27 14:13:06] 桶 L1_UniqueCarrier=EV 子桶样本不足（最小子桶 n=2 < 200），不再细分
[INFO][BT][2025-11-27 14:13:06] 桶 L1_UniqueCarrier=FL 子桶样本不足（最小子桶 n=11 < 200），不再细分
[INFO][BT][2025-11-27 14:13:06] 桶 L1_UniqueCarrier=HP 子桶样本不足（最小子桶 n=6 < 200），不再细分
[INFO][BT][2025-11-27 14:13:06] 桶 L1_UniqueCarrier=MQ 子桶样本不足（最小子桶 n=4 < 200），不再细分
[INFO][BT][2025-11-27 14:13:06] 桶 L1_UniqueCarrier=NW 子桶样本不足（最小子桶 n=2 < 200），不再细分
[INFO][BT][2025-11-27 14:13:06] 桶 L1_UniqueCarrier=OH 子桶样本不足（最小子桶 n=1 < 200），不再细分
[INFO][BT][2025-11-27 14:13:06] 桶 L1_UniqueCa

,model,Precision_mean,Precision_std,Recall_mean,Recall_std,F1_mean,F1_std,BAC_mean,BAC_std,AUC_mean,...,MCC_mean,MCC_std,Kappa_mean,Kappa_std,BND_ratio_mean,BND_ratio_std,POS_Coverage_mean,POS_Coverage_std,Regret_mean,Regret_std
0,BTTWD,0.245040,0.001690,0.617349,0.003069,0.350825,0.001931,0.633074,0.001895,0.684313,...,0.197688,0.002951,0.164352,0.002796,0.20522,0.003884,0.058961,0.001331,0.681251,0.001244
1,XGBoost,0.277064,0.000987,0.468221,0.001800,0.348127,0.001196,0.621323,0.000954,0.684799,...,0.199814,0.001570,0.189378,0.001497,0.00000,0.000000,NaN,NaN,0.604833,0.001504


【INFO】【2025-11-27 14:13:21】【步骤6摘要】BTTWD 与基线的 k 折结果已生成并保存。


In [8]:
# 步骤7：桶级别分析
bucket_metrics_path = os.path.join(root_path, cfg['OUTPUT']['results_dir'], 'bucket_metrics.csv')
if os.path.exists(bucket_metrics_path):
    bucket_metrics_df = pd.read_csv(bucket_metrics_path)
    display(bucket_metrics_df.head())
    bucket_metrics_df.plot(x='bucket_id', y='pos_rate_all', kind='bar', figsize=(12,4), title='桶正类比例')
    plt.ylabel('正类比例')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig(fig_bucket, bbox_inches='tight')
    plt.close()
log_info('【步骤7摘要】桶级指标（如存在）已整理，可用于局部化分析。')


,bucket_id,layer,parent_bucket_id,n_train,n_val,pos_rate_train,pos_rate_val,alpha,beta,regret_val,...,threshold_n_samples,n_all,pos_rate_all,n_test,pos_rate_test,BND_ratio_test,POS_Coverage_test,regret_test,fold,pos_rate
0,L1_UniqueCarrier=WN,L1,NaN,78821,33486,0.178531,0.178254,0.3,0.2,0.676656,...,33486,112307,0.178448,NaN,NaN,NaN,NaN,NaN,1,0.178448
1,L1_UniqueCarrier=DL,L1,NaN,72439,31133,0.133795,0.137443,0.3,0.2,0.634520,...,31133,103572,0.134892,NaN,NaN,NaN,NaN,NaN,1,0.134892
2,L1_UniqueCarrier=AA,L1,NaN,63919,27337,0.146279,0.148370,0.3,0.2,0.674050,...,27337,91256,0.146905,NaN,NaN,NaN,NaN,NaN,1,0.146905
3,L1_UniqueCarrier=US,L1,NaN,58780,25178,0.143756,0.142625,0.3,0.2,0.650032,...,25178,83958,0.143417,NaN,NaN,NaN,NaN,NaN,1,0.143417
4,L1_UniqueCarrier=UA,L1,NaN,55374,23967,0.170026,0.170735,0.3,0.2,0.757813,...,23967,79341,0.170240,NaN,NaN,NaN,NaN,NaN,1,0.170240


【INFO】【2025-11-27 14:13:22】【步骤7摘要】桶级指标（如存在）已整理，可用于局部化分析。


In [9]:
# 步骤8：结果汇总
log_info('【步骤8】检查结果文件与图表。')
results_dir = os.path.join(root_path, cfg['OUTPUT']['results_dir'])
figs_dir = os.path.join(root_path, cfg['OUTPUT']['figs_dir'])
os.makedirs(results_dir, exist_ok=True)
os.makedirs(figs_dir, exist_ok=True)
print(os.listdir(results_dir))
print(os.listdir(figs_dir))
log_info('【全部步骤完成】Airlines 数据集的 BT-TWD 实验结束。')


【INFO】【2025-11-27 14:13:23】【步骤8】检查结果文件与图表。
['bucket_metrics.csv', 'bucket_thresholds_per_fold.csv', 'metrics_kfold_per_fold.csv', 'metrics_kfold_summary.csv']
['bank_class_distribution.png', 'bucket_metrics_bar.png', 'class_distribution.png', 'metrics_compare.png']
【INFO】【2025-11-27 14:13:23】【全部步骤完成】Airlines 数据集的 BT-TWD 实验结束。
